<a href="https://colab.research.google.com/github/zh1212121/NLP-Assn-2/blob/main/Assignment_2_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/zh1212121/NLP-Assn-2.git

Cloning into 'NLP-Assn-2'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 30 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (30/30), done.


##1. Import libraries and check version

In [1]:
!pip install scikit-learn==0.21
!pip uninstall numpy -y
!pip install numpy==1.19.0
!pip install nltk

Uninstalling numpy-1.19.0:
  Successfully uninstalled numpy-1.19.0
  Using cached https://files.pythonhosted.org/packages/a8/31/e2c3eda7afe7dab08e1f24767b8e38ff2f30dc82bd74aa3a5324c550366a/numpy-1.19.0-cp37-cp37m-manylinux2010_x86_64.whl
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.19.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [2]:
import numpy as np
import pandas as pd
import torch
import re
import string
import nltk
import sklearn as sk
import matplotlib.pyplot as plt
import time
import pickle

from sklearn import model_selection, metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.svm import SVC
from sklearn.utils import shuffle


from google.colab import files

In [3]:
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
seed = 1234

In [5]:
print(np.__version__)
print(pd.__version__)
print(sk.__version__)

1.19.0
1.1.5
0.21.0


##2. Import dataset

In [7]:
#input the test dataset path here
path_exam = "https://raw.githubusercontent.com/zh1212121/NLP-Assn-2/main/Test-format.csv"

In [9]:
#import given datasets
exam_raw = pd.read_csv(path_exam, header = None)

In [10]:
exam_raw.head()

,0
0,review 1 x y z
1,review 2 x y z
2,review 3 x y z
3,review 4 x y z
4,review 5 x y z


## 3. Data preprocessing

In [11]:
#change text to lower case
def convert_to_lower(text):
  return text.lower()

In [12]:
#remove <br />
def remove_newline(text):
  return re.sub('<br />', '', text)

In [13]:
#remove http:// links
def remove_link(text):
  return re.sub(r'http[s]?://\S+\b', '', text)

In [14]:
#decontraction
def decontract(text):
  text = re.sub(r"won\'t", " will not", text)
  text = re.sub(r"won\'t've", " will not have", text)
  text = re.sub(r"can\'t", " can not", text)
  text = re.sub(r"don\'t", " do not", text)
    
  text = re.sub(r"can\'t've", " can not have", text)
  text = re.sub(r"ma\'am", " madam", text)
  text = re.sub(r"let\'s", " let us", text)
  text = re.sub(r"ain\'t", " am not", text)
  text = re.sub(r"shan\'t", " shall not", text)
  text = re.sub(r"sha\n't", " shall not", text)
  text = re.sub(r"o\'clock", " of the clock", text)
  text = re.sub(r"y\'all", " you all", text)

  text = re.sub(r"n\'t", " not", text)
  text = re.sub(r"n\'t've", " not have", text)
  text = re.sub(r"\'re", " are", text)
  text = re.sub(r"\'s", " is", text)
  text = re.sub(r"\'d", " would", text)
  text = re.sub(r"\'d've", " would have", text)
  text = re.sub(r"\'ll", " will", text)
  text = re.sub(r"\'ll've", " will have", text)
  text = re.sub(r"\'t", " not", text)
  text = re.sub(r"\'ve", " have", text)
  text = re.sub(r"\'m", " am", text)
  text = re.sub(r"\'re", " are", text)
  return text

#reference:
#https://www.kaggle.com/faressayah/sentiment-model-with-tensorflow-transformers

In [15]:
#remove punctuation
print(string.punctuation)

def remove_punc(text):
  return text.translate(str.maketrans('', '', string.punctuation))

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [16]:
#remove special characters & numbers 
def remove_special(text, remove_punc = True, remove_digits=True):
  if remove_punc:
    if remove_digits:
      pattern=r'[^a-zA-z\s]'
      text=re.sub(pattern,'',text)
    else:
      pattern=r'[^a-zA-z0-9\s]'
      text=re.sub(pattern,'',text)
  else:
    if remove_digits:
      pattern=r'[^a-zA-z\!\"#$%&\'()*+,-.//:;<=>\?@[\]^_`{|}~\s'
      text=re.sub(pattern,'',text)
    else:
      pattern=r'[^a-zA-z0-9\!\"#$%&\'()*+,-.//:;<=>\?@[\]^_`{|}~\s'
      text=re.sub(pattern,'',text)
  return text

In [17]:
#view stopword list
stop = stopwords.words('english')
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [18]:
#define my own stopword list to exclude those stopwards that could change meaning of a sentence, eg. can't, doesn't
stopword_list = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", 
                 "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 
                 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 
                 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 
                 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 
                 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 
                 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 
                 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 
                 'up', 'down', 'in', 'out', 'on', 'off', 'further', 'then', 'here', 'there', 'when', 'where', 
                 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more','other', 'some', 'such', 'own', 
                 'same', 'so', 'than', 'too', 's', 't', 'can', 'will', 'just', 'now', 'd', 'll', 'm', 'o', 
                 're', 've', 'y', ]
print(stopword_list)

def remove_stopword(text):
  result = ' '.join([i for i in text.split() if not i in stopword_list])
  return result

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'further', 'then', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'other', 'some', 'such', 'own', 'same', 'so', 'than', 'too', 's', 't', 'can', 'will', 'just', 'now', 'd', 'll', 'm', 'o', 're', 've', 'y

In [19]:
#Lemmatization
def lemmatize(text):
  lemmatizer = WordNetLemmatizer()
  tokens = word_tokenize(text)
  result = ' '.join([lemmatizer.lemmatize(w) for w in tokens])
  return result

In [20]:
#remove single character as it generally has less meaning
def remove_single_char(text):
  return re.sub(r'\b\w\b', '', text)

In [21]:
#reduce multiple whitespaces to single space
def reduce_space(text):
  return re.sub(r'\s+', ' ', text).strip()

In [22]:
def clean_text(text, remove_punc = False, remove_digits = False):
  text = convert_to_lower(text)
#  print(text)
  text = remove_newline(text)
#  print(text)
  text = remove_link(text)
#  print(text)
  text = decontract(text)
#  print(text)
  text = remove_special(text, remove_punc, remove_digits)
#  print(text)
  text = remove_stopword(text)
#  print(text)  
#  text = lemmatize(text)
#  print(text)
  text = remove_single_char(text)
#  print(text)
  text = reduce_space(text)
#  print(text)

  return text

In [23]:
punc = True
dig = False

In [24]:
exam_raw['Clean_text'] = exam_raw[:][0].apply(clean_text, remove_punc = punc, remove_digits = dig)

In [25]:
exam_raw.head()

,0,Clean_text
0,review 1 x y z,review
1,review 2 x y z,review
2,review 3 x y z,review
3,review 4 x y z,review
4,review 5 x y z,review


In [26]:
x_exam = exam_raw['Clean_text']

##4. Load model

In [27]:
filename_vec = '/content/NLP-Assn-2/vectorizer.pickle'
pickle.load(open(filename_vec, 'rb'))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.8, max_features=None,
                min_df=5, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=True, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [28]:
filename = '/content/NLP-Assn-2/model.sav'
model = pickle.load(open(filename, 'rb'))

##5. Make Prediction

In [35]:
y_pred = model.predict(x_exam)
#print(y_pred)

In [32]:
#output to df
output = pd.DataFrame({'Text': x_exam, 'Pred_Label': y_pred})
output.head()

,Text,Pred_Label
0,review,0
1,review,0
2,review,0
3,review,0
4,review,0


In [ ]:
#Download the Predicted result
output.to_csv('Output_ZhengWanyu.csv')
files.download('Output_ZhengWanyu.csv')

#Evaluation - **Require User Input**

In [ ]:
#define the ground labels as y_exam
y_exam = 

##Evaluation

In [34]:
print('classification report on test data is shown below:')
report = classification_report(y_exam, y_pred)
print(report)

f1_score = sk.metrics.f1_score(y_exam, y_pred)
print('f1 score of the model on test data is:', round(f1_score, 2))
print(f1_score)

accuracy = sk.metrics.accuracy_score(y_exam, y_pred)
print('accuracy of the model on test data is:', round(accuracy,2))
print(accuracy)

classification report on test data is shown below:


NameError: ignored